# Étude du cas de Sod (tube à choc)

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
plt.rcParams['font.size'] = 14 ; plt.rcParams['axes.grid'] = True ; plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['lines.linewidth'] = 1.5 ; plt.rcParams['legend.fontsize'] = 12
#
import flowdyn.mesh  as mesh
import flowdyn.modelphy.euler as euler
import flowdyn.modeldisc      as modeldisc
import flowdyn.field as data
from flowdyn.xnum        import *
from flowdyn.integration import *
import flowdyn.solution.euler_riemann as sol
#print plt.rcParams


## Définition des maillages, du modèle physique et solution initiales


In [2]:
meshsim  = mesh.unimesh(ncell=200,  length=10., x0=-4.)
meshref  = mesh.unimesh(ncell=1000, length=10., x0=-4.)

model1 = euler.model()
model2 = euler.model()
sod   = sol.Sod_subsonic(model1) # sol.Sod_supersonic(model1) # 

bcL  = { 'type': 'dirichlet',  'prim':  sod.bcL() }
bcR  = { 'type': 'dirichlet',  'prim':  sod.bcR() }
xnum1 = muscl(minmod) # 
xnum2 = muscl(vanalbada) # 

rhs1 = modeldisc.fvm(model1, meshsim, xnum1, numflux='hllc', bcL=bcL, bcR=bcR)
solver1 = rk3ssp(meshsim, rhs1)
rhs2 = modeldisc.fvm(model2, meshsim, xnum2, numflux='hllc', bcL=bcL, bcR=bcR)
solver2 = rk3ssp(meshsim, rhs2)

bcL  = { 'type': 'dirichlet',  'prim':  sod.bcL() }
bcR  = { 'type': 'dirichlet',  'prim':  sod.bcR() }


In [3]:
# computation
#
endtime = 2.8
cfl     = 1.

finit = sod.fdata(meshsim)

fsol1 = solver1.solve(finit, cfl, [endtime])
solver1.show_perf()
fsol2 = solver2.solve(finit, cfl, [endtime])
solver2.show_perf()

# Figure / Plot

fref = sod.fdata(meshref, endtime)

for name in ['density', 'pressure', 'mach']:
    fig = plt.figure(figsize=(10,8))
    plt.ylabel(name)
    plt.grid(linestyle='--', color='0.5')
    #finit.plot(name, 'k-.')
    fref.plot(name, 'k-')
    fsol1[0].plot(name, 'b-')
    fsol2[0].plot(name, 'r-')
    fig.savefig(name+'.png', bbox_inches='tight')


## Calcul et comparaison

In [4]:
from IPython.core.display import HTML ; HTML(open("../custom.css", "r").read()) # notebook style